In [5]:
import time
import json
import numpy as np
import pandas as pd
import requests
import networkx as nx

from statistics import mean 
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler

In [44]:
tr_list = ['c7a3673f869fc9c5c5e8f032af97546efed0435762e4ef728f99b6725c027f9b',
           '34ed08b17fff9cbe51ce33c62c6c1bb115f82aeec60997bfb216126bf747b5c8']

addr_list = ['3AuShAzkBaAvL6Xc4y61Jfnf7AM7s3Wpmb',
             '1JqmxxswaEpvEcLJJLe8BTvmSHKYzY7LBn', 
             '3HjJQWMhhJjAkPiZKywDYyhXDdWkSFLW9F', 
             '1KqtK7K46kiFCV2XzAt43AYxNVwPuEccW6',
             '1HRPPH6gVPhL4tTbAM5CCcDFt2bXpR5vNm',
             '1JqmxxswaEpvEcLJJLe8BTvmSHKYzY7LBn',
             '3BPMdL5WVNYdBWVzfqRNkAMHiMkegporXn',
             '3MycntSknbumLyRX56X8xbwxJ1zqHHq1VD',
             '3HpWfb94RXgbj59PnJijayj9t2ebFkfCQn',
             '3GdH7YrcFgQv4EANKDw66aZwruYCQ21LpL',
             '1FQFFxz5TPJsRwkqHJa4ecd9gjQS2hYcbt',
             '3HjJQWMhhJjAkPiZKywDYyhXDdWkSFLW9F',
             '1KqtK7K46kiFCV2XzAt43AYxNVwPuEccW6']

In [45]:
%%time
with open('add_json.json', 'r') as f:
    results = json.load(f)

CPU times: user 49.9 ms, sys: 7.72 ms, total: 57.6 ms
Wall time: 58.6 ms


In [46]:
df = []
trs = list(results.keys())
for tr in trs:
    df.append(results[tr])
    
cols = list(results[trs[0]].keys())
data = pd.DataFrame(df, columns=cols)

In [47]:
data['is_coinbase'] = data['is_coinbase'].map({False: 0, True: 1}).astype(int)
data['is_double_spend'] = data['is_double_spend'].map({False: 0, True: 1}).astype(int)
data['is_sw_tx'] = data['is_sw_tx'].map({False: 0, True: 1}).astype(int)

In [48]:
#add suspicious labels
data['is_suspicious'] = data['txid'].apply(lambda x: 1 if x in tr_list else 0)

## Feature Matrix

Хотим получить матрицу, где по строкам будут транзакции относительно адресов:
1. если у транзакции было n входящих (исходящих) адресов, то данная транзакция преобразовывается в n строк
2. если у транзакции было n входящих и m исходящих адресов, то данная транзакция преобразовывается в n+m строк

#### Features

1. 'txid' - id текущей транзакции
2. 'confirmations' - число подтверждений 
3. 'time' - время совершения данной транзакции 
4. 'is_coinbase' - флаг 
5. 'is_double_spend' - флаг
6. 'is_sw_tx' - флаг 
7. 'weight' - вес
8. 'vsize' - вес
9. 'inputs_count' - число входящих адресов
10. 'outputs_count'- число исходящих адресов
11. 'inputs_value' - общая сумма текущей транзакции (отличается от curr_value, если более 1 входа)
12. 'outputs_value' - общая сумма исходящей транзакции (отличается от curr_value-fee, если более 1 выхода) 
13. 'max_prev_val' - максимальная сумма среди предыдущих транзакций (отличается от tot_prev_tx_value, если более 1 входа) 
14. 'min_prev_val' - минимальная сумма среди предыдущих транзакций (отличается от tot_prev_tx_value, если более 1 входа)   
15. 'avg_prev_val' - средняя сумма предыдущих транзакций (отличается от tot_prev_tx_value, если более 1 входа)          
16. 'max_next_val' - максимальная сумма среди следующих транзакций (отличается от tot_next_tx_value, если более 1 входа)
17. 'min_next_val' - минимальная сумма среди следующих транзакций (отличается от tot_next_tx_value, если более 1 входа)
18. 'avg_next_val' - средняя сумма следующих транзакций (отличается от tot_next_tx_value, если более 1 входа)          
19. 'diff_max_prev_time' - максимальная разница во времени текущей транзакции относительно всех предыдущих (отличается от diff_prev_time, если более 1 входа)
20. 'diff_min_prev_time' - минимальна разница во времени текущей транзакции относительно всех предыдущих (отличается от diff_prev_time, если более 1 входа)
21. 'diff_avg_prev_time' - средняя разница во времени текущей транзакции относительно всех предыдущих (отличается от diff_prev_time, если более 1 входа)
22. 'diff_max_next_time' - максимальная разница во времени текущей транзакции относительно всех следующих (отличается от diff_next_time, если более 1 входа)
23. 'diff_min_next_time' - минимальная разница во времени текущей транзакции относительно всех следующих (отличается от diff_next_time, если более 1 входа)
24. 'diff_avg_next_time' - средняя разница во времени текущей транзакции относительно всех следующих (отличается от diff_next_time, если более 1 входа)
25. 'is_suspicious' - флаг, является ли транзакция подозрительной (1, если транзакция есть в первоначальном списке tr_list)
26. 'transaction_type' - тип пранзакции (1,2,3,4)
27. 'input_add' - входящий адрес
28. 'output_add' - исходящий адрес
29. 'curr_value' - сумма транзакции для данного входящего/исходящего адреса (в зависимости от transaction_type)
30. 'tot_prev_tx_value' - размер предыдущей транзакции для данного input_add (если transaction_type=4, то берется avg_prev_val) <span style="color:red"> (лучше брать avg, min или max ??????)</span>
31. 'tot_next_tx_value' - размер следующей транзакции для данного input_add (если transaction_type=4, то берется avg_next_val) <span style="color:red"> (лучше брать avg, min или max ??????)</span>
32. 'diff_prev_time' - задержка текущей транзакции относительно предыдущей (если transaction_type=4, то берется diff_avg_prev_time)  <span style="color:red"> (лучше брать avg, min или max ??????)</span>
33. diff_next_time - задержка следующей транзакции относительно текущей (если transaction_type=4, то берется diff_avg_next_time)  <span style="color:red"> (лучше брать avg, min или max ??????)</span>

In [49]:
data.head()

,txid,confirmations,time,is_coinbase,is_double_spend,is_sw_tx,weight,vsize,inputs_count,outputs_count,...,min_next_val,avg_next_val,list_of_outp_adds,diff_max_prev_time,diff_min_prev_time,diff_avg_prev_time,diff_max_next_time,diff_min_next_time,diff_avg_next_time,is_suspicious
0,169a95aac5b96d349aa7b20674867562e9c6a7c84980eb...,3981,1556228078,0,0,1,36509,9128,100,1,...,138336833.0,138336833.0,[{'prev_address': '3JLTRQptT4FaPZzru5cAYxMQPdQ...,11559,573784,76896,48535.0,48535.0,48535.0,0
1,cb2b13b7a4cf99c4ad3fc97b28f12970a350ffed0d3459...,4147,1556129730,0,0,1,36516,9129,100,1,...,207089082.0,207089082.0,[{'prev_address': '3KRgrPeHyTgAPgcVMrRwmm81BFZ...,6109,387664,119119,225962.0,225962.0,225962.0,0
2,f0e6da77a308555a71e6d47fdee240f344ee0db809354f...,4547,1555878831,0,0,1,36519,9130,100,1,...,106531671.0,106531671.0,[{'prev_address': '35tsuFKUnBLCS6PWgGcpc7WjG4r...,30498,291776,88164,87037.0,87037.0,87037.0,0
3,98a667ecdb57d4d83d14bf66532746a76ca69a287d21f6...,4548,1555878579,0,0,1,36516,9129,100,1,...,91608129.0,91608129.0,[{'prev_address': '3Ck1WUMXJXegw8uvEAQ33oiF2wA...,23268,187262,61744,42123.0,42123.0,42123.0,0
4,37515761a5a826a2602402f2893da1b0b5ff9f217625c0...,4549,1555878451,0,0,1,3942,986,10,2,...,23624666.0,32883110.0,[{'prev_address': '1NoAoAJspDrAXqFW5ySqUaUg8LS...,28245,191202,45454,6030.0,380.0,3205.0,0


In [50]:
data['list_of_outp_adds'][0][0]

{'prev_address': '3JLTRQptT4FaPZzru5cAYxMQPdQRFTtHia',
 'curr_value': 125367776,
 'next_time': 1556276613,
 'total_next_val': 138336833}

Будем рассматривать 4 вида транзакций (и по-разному формировать для них матрицу фичей)

|transaction_type|num_of_inputs  |num_of_outputs|
|----|---------------|--------------|
| 1  | 1             | 1            |
| 2  | several       | 1            |
| 3  | 1             | several      |
| 4  | several       | several      |

In [51]:
def transaction_type(row):
    if   row[0]==1 and row[1]==1: return 1
    elif row[0] >1 and row[1]==1: return 2
    elif row[0]==1 and row[1] >1: return 3
    elif row[0] >1 and row[1] >1: return 4
    else: return 0

In [52]:
data['transaction_type'] = [transaction_type(x) for x in data[['inputs_count', 'outputs_count']].values]

In [53]:
#кол-во транзакций каждого типа в датасете
data.groupby('transaction_type').count()[['txid']].T

transaction_type,1,2,3,4
txid,28,92,56,180


In [54]:
cols = ['txid',
 'confirmations',
 'time',
 'is_coinbase',
 'is_double_spend',
 'is_sw_tx',
 'weight',
 'vsize',
 'inputs_count',
 'outputs_count',
 'inputs_value',
 'outputs_value',
 'max_prev_val',
 'min_prev_val',
 'avg_prev_val',
 'max_next_val',
 'min_next_val',
 'avg_next_val',
 'diff_max_prev_time',
 'diff_min_prev_time',
 'diff_avg_prev_time',
 'diff_max_next_time',
 'diff_min_next_time',
 'diff_avg_next_time',
 'is_suspicious',
 'transaction_type',
 'input_add',
 'output_add',
 'curr_value',
 'tot_prev_tx_value',
 'tot_next_tx_value',
 'diff_prev_time',
 'diff_next_time']

## transaction_type = 1

In [55]:
data_1 = data[data['transaction_type'] == 1]

#входящий и исходящий адреса
data_1['input_add'] = data_1['list_of_inp_adds'].apply(lambda x: x[0]['prev_address'])
data_1['output_add'] = data_1['list_of_outp_adds'].apply(lambda x: x[0]['prev_address'])

#сумма текущей транзакции
#считаем current value по input'у
data_1['curr_value'] = data_1['list_of_inp_adds'].apply(lambda x: x[0]['curr_value'])

#общая сумма предыдущей и следующей транзакций (не сколько на адрес пришло, а именно всей транзакции целиком)
data_1['tot_prev_tx_value'] = data_1['list_of_inp_adds'].apply(lambda x: x[0]['total_prev_val'])
data_1['tot_next_tx_value'] = data_1['list_of_outp_adds'].apply(lambda x: x[0]['total_next_val'])

#дата совершения предыдущей и следующей транзакций 
data_1['prev_time'] = data_1['list_of_inp_adds'].apply(lambda x: x[0]['prev_time'])
data_1['next_time'] = data_1['list_of_outp_adds'].apply(lambda x: x[0]['next_time'])

#задержка относительно предыдущей и следующей транзакций 
data_1['diff_prev_time'] = data_1['time'] - data_1['prev_time']
data_1['diff_next_time'] = data_1['next_time'] - data_1['time']


data_1_upd = data_1[cols]

In [56]:
len(data_1_upd.columns)

33

## transaction_type = 2

In [57]:
data_2 = data[data['transaction_type'] == 2].reset_index(drop=True)


#convert input adds
tmp_inp = data_2.list_of_inp_adds\
          .apply(pd.Series).merge(data_2[['txid','list_of_inp_adds']], left_index = True, right_index = True)\
          .drop(['list_of_inp_adds'], axis = 1)\
          .melt(id_vars = ['txid'], value_name = 'input_add')\
          .drop('variable', axis = 1).dropna().reset_index(drop=True)

tmp_inp = tmp_inp['input_add'].apply(pd.Series)\
          .merge(tmp_inp[['txid','input_add']], left_index = True, right_index = True)\
          .drop(['input_add'], axis = 1)\
          .rename({'prev_address': 'input_add','total_prev_val': 'tot_prev_tx_value'}, axis=1)


#вытаскиваем исходящий адрес
data_2['output_add'] = data_2['list_of_outp_adds'].apply(lambda x: x[0]['prev_address'])
#вытаскиваем общую сумму следующей транзакции 
data_2['tot_next_tx_value'] = data_2['list_of_outp_adds'].apply(lambda x: x[0]['total_next_val'])
#дата совершения следующей транзакции
data_2['next_time'] = data_2['list_of_outp_adds'].apply(lambda x: x[0]['next_time'])

data_2 = data_2.set_index('txid')
tmp_inp = tmp_inp.set_index('txid')


cols_to_take = ['output_add', 'tot_next_tx_value', 'next_time', 'confirmations',
                'time', 'is_coinbase', 'is_double_spend', 'is_sw_tx', 'weight', 'vsize',
                'inputs_count', 'outputs_count', 'inputs_value', 'outputs_value', 'max_prev_val',
                'min_prev_val', 'avg_prev_val', 'max_next_val', 'min_next_val', 'avg_next_val',
                'diff_max_prev_time', 'diff_min_prev_time', 'diff_avg_prev_time', 'diff_max_next_time',
                'diff_min_next_time', 'diff_avg_next_time', 'is_suspicious', 'transaction_type']

data_2_upd = tmp_inp.join(data_2[cols_to_take], how='left').reset_index()

In [58]:
#задержка относительно предыдущей и следующей транзакций 
data_2_upd['diff_prev_time'] = data_2_upd['time'] - data_2_upd['prev_time']
data_2_upd['diff_next_time'] = data_2_upd['next_time'] - data_2_upd['time']

In [59]:
data_2_upd = data_2_upd[cols]

In [60]:
len(data_2_upd.columns)

33

## transaction_type = 3

In [61]:
data_3 = data[data['transaction_type'] == 3].reset_index(drop=True)

#convert input adds
tmp_outp = data_3.list_of_outp_adds\
          .apply(pd.Series).merge(data_3[['txid','list_of_outp_adds']], left_index = True, right_index = True)\
          .drop(['list_of_outp_adds'], axis = 1)\
          .melt(id_vars = ['txid'], value_name = 'output_add')\
          .drop('variable', axis = 1).dropna().reset_index(drop=True)

tmp_outp = tmp_outp['output_add'].apply(pd.Series)\
          .merge(tmp_outp[['txid','output_add']], left_index = True, right_index = True)\
          .drop(['output_add'], axis = 1)\
          .rename({'prev_address': 'output_add','total_next_val': 'tot_next_tx_value'}, axis=1)

#вытаскиваем исходящий адрес
data_3['input_add'] = data_3['list_of_inp_adds'].apply(lambda x: x[0]['prev_address'])
#вытаскиваем общую сумму следующей транзакции 
data_3['tot_prev_tx_value'] = data_3['list_of_inp_adds'].apply(lambda x: x[0]['total_prev_val'])
#дата совершения следующей транзакции
data_3['prev_time'] = data_3['list_of_inp_adds'].apply(lambda x: x[0]['prev_time'])

data_3 = data_3.set_index('txid')
tmp_outp = tmp_outp.set_index('txid')


cols_to_take = ['input_add', 'tot_prev_tx_value', 'prev_time', 'confirmations',
                'time', 'is_coinbase', 'is_double_spend', 'is_sw_tx', 'weight', 'vsize',
                'inputs_count', 'outputs_count', 'inputs_value', 'outputs_value', 'max_prev_val',
                'min_prev_val', 'avg_prev_val', 'max_next_val', 'min_next_val', 'avg_next_val',
                'diff_max_prev_time', 'diff_min_prev_time', 'diff_avg_prev_time', 'diff_max_next_time',
                'diff_min_next_time', 'diff_avg_next_time', 'is_suspicious', 'transaction_type']

data_3_upd = tmp_outp.join(data_3[cols_to_take], how='left').reset_index()

In [62]:
#задержка относительно предыдущей и следующей транзакций 
data_3_upd['diff_prev_time'] = data_3_upd['time'] - data_3_upd['prev_time']
data_3_upd['diff_next_time'] = data_3_upd['next_time'] - data_3_upd['time']

In [63]:
data_3_upd = data_3_upd[cols]

## transaction_type = 4

In [64]:
data_4 = data[data['transaction_type'] == 4].reset_index(drop=True)

In [65]:
#convert input adds
tmp_inp = data_4.list_of_inp_adds\
          .apply(pd.Series).merge(data_4[['txid','list_of_inp_adds']], left_index = True, right_index = True)\
          .drop(['list_of_inp_adds'], axis = 1)\
          .melt(id_vars = ['txid'], value_name = 'input_add')\
          .drop('variable', axis = 1).dropna().reset_index(drop=True)

tmp_inp = tmp_inp['input_add'].apply(pd.Series)\
          .merge(tmp_inp[['txid','input_add']], left_index = True, right_index = True)\
          .drop(['input_add'], axis = 1)\
          .rename({'prev_address': 'input_add','total_prev_val': 'tot_prev_tx_value'}, axis=1)

tmp_inp['output_add'] = tmp_inp['txid']

In [66]:
#convert input adds
tmp_outp = data_4.list_of_outp_adds\
          .apply(pd.Series).merge(data_4[['txid','list_of_outp_adds']], left_index = True, right_index = True)\
          .drop(['list_of_outp_adds'], axis = 1)\
          .melt(id_vars = ['txid'], value_name = 'output_add')\
          .drop('variable', axis = 1).dropna().reset_index(drop=True)

tmp_outp = tmp_outp['output_add'].apply(pd.Series)\
          .merge(tmp_outp[['txid','output_add']], left_index = True, right_index = True)\
          .drop(['output_add'], axis = 1)\
          .rename({'prev_address': 'output_add','total_next_val': 'tot_next_tx_value'}, axis=1)

tmp_outp['input_add'] = tmp_outp['txid']

In [67]:
data_4['tot_next_tx_value'] = data_4['avg_next_val']
data_4['tot_prev_tx_value'] = data_4['avg_prev_val']

In [68]:
data_4['next_time'] = data_4['avg_next_time']
data_4['prev_time'] = data_4['avg_prev_time']

In [69]:
data_4 = data_4.set_index('txid')
tmp_inp = tmp_inp.set_index('txid')
tmp_outp = tmp_outp.set_index('txid')

In [70]:
cols_to_take_1 = ['tot_next_tx_value', 'next_time', 'confirmations',
                'time', 'is_coinbase', 'is_double_spend', 'is_sw_tx', 'weight', 'vsize',
                'inputs_count', 'outputs_count', 'inputs_value', 'outputs_value', 'max_prev_val',
                'min_prev_val', 'avg_prev_val', 'max_next_val', 'min_next_val', 'avg_next_val',
                'diff_max_prev_time', 'diff_min_prev_time', 'diff_avg_prev_time', 'diff_max_next_time',
                'diff_min_next_time', 'diff_avg_next_time', 'is_suspicious', 'transaction_type']

cols_to_take_2 = ['tot_prev_tx_value', 'prev_time', 'confirmations',
                'time', 'is_coinbase', 'is_double_spend', 'is_sw_tx', 'weight', 'vsize',
                'inputs_count', 'outputs_count', 'inputs_value', 'outputs_value', 'max_prev_val',
                'min_prev_val', 'avg_prev_val', 'max_next_val', 'min_next_val', 'avg_next_val',
                'diff_max_prev_time', 'diff_min_prev_time', 'diff_avg_prev_time', 'diff_max_next_time',
                'diff_min_next_time', 'diff_avg_next_time', 'is_suspicious', 'transaction_type']

In [71]:
data_4_upd_1 = tmp_inp.join(data_4[cols_to_take_1], how='left').reset_index()
data_4_upd_2 = tmp_outp.join(data_4[cols_to_take_2], how='left').reset_index()

In [72]:
#задержка относительно предыдущей и следующей транзакций 
data_4_upd_1['diff_prev_time'] = data_4_upd_1['time'] - data_4_upd_1['prev_time']
data_4_upd_1['diff_next_time'] = data_4_upd_1['next_time'] - data_4_upd_1['time']

#задержка относительно предыдущей и следующей транзакций 
data_4_upd_2['diff_prev_time'] = data_4_upd_2['time'] - data_4_upd_2['prev_time']
data_4_upd_2['diff_next_time'] = data_4_upd_2['next_time'] - data_4_upd_2['time']

In [73]:
data_4_upd_1 = data_4_upd_1[cols]
data_4_upd_2 = data_4_upd_2[cols]

## Cобираем все вместе

In [74]:
print(f'data_1_upd shape: {data_1_upd.shape}')
print(f'data_2_upd shape: {data_2_upd.shape}')
print(f'data_3_upd shape: {data_3_upd.shape}')
print(f'data_4_upd_1 shape: {data_4_upd_1.shape}')
print(f'data_4_upd_2 shape: {data_4_upd_2.shape}')

data_1_upd shape: (28, 33)
data_2_upd shape: (7322, 33)
data_3_upd shape: (213, 33)
data_4_upd_1 shape: (10023, 33)
data_4_upd_2 shape: (1191, 33)


In [75]:
result_df = pd.concat([data_1_upd, data_2_upd, data_3_upd, data_4_upd_1, data_4_upd_2]).reset_index(drop=True)
#result_df = pd.concat([data_2_upd, data_4_upd_1, data_4_upd_2]).reset_index(drop=True)

In [84]:
#пометим искуственно сгенерированные inputs and outputs (когда связь многие ко многим, и создаю доп.узел)
result_df['true_input'] = result_df['input_add'].apply(lambda x: 1 if len(x)<=40 else 0)
result_df['true_output'] = result_df['output_add'].apply(lambda x: 1 if len(x)<=40 else 0)

,curr_value,tot_prev_tx_value,diff_prev_time
input_add,,,
02f082f82aa9a019171863b99bbb02ff66e52954,11223736,171851132,9114.0
044193591d7cc88f0ad7483f0e22c0c3947212ca90f4d680539e2b8f88e56cf6,873597343,11036572344,3507.0
077b9ce453b08086972f33b2f2b6273117275aeefce7f240d138a8f96394886e,4944263,60497416,10240.0
078b8e225449a2baa66ecc06798a43a7b714faf405777ffac9b450314cdf4d5c,315965157,18758237456,10506.0
07ce5ec412b7fc7b76e8ecaa8239ea9a72f4ed55a8c7d0ae9145d3cb64588ae3,833468792,15121055328,566829.0


Хочу сделать датафрейм, где 1 строка = 1 адрес + аггрегированные значения признаков по нему

1. count_as_inp_add - число транзакций, где текущий адрес - input
2. count_as_outp_add - число транзакций, где текущий адрес - output
3. count_as_inp_outp_add - число транзакций, где текущий адрес - input и output

4. is_coinbase - sum (если больше нуля, то была 1)
5. is_double_spend - sum (если больше нуля, то была 1)
6. is_sw_tx	 - sum (если больше нуля, то была 1)
7. weight
    * min
    * max
    * mean
    * mode
    * std
8. vsize
    * min
    * max
    * mean
    * mode
    * std
9. inputs_count
    * min
    * max
    * mean
    * mode
    * std
    * sum
10. outputs_count
    * min
    * max
    * mean
    * mode
    * std
    * sum
11. inputs_value
    * min
    * max
    * mean
    * mode
    * std
    * sum
12. max_prev_val, min_prev_val, avg_prev_val, max_next_val, min_next_val, avg_next_val, diff_max_prev_time, diff_min_prev_time, diff_avg_prev_time, diff_max_next_time, diff_min_next_time, diff_avg_next_time, tot_prev_tx_value, tot_next_tx_value, diff_prev_time, diff_next_time
    * min
    * max
    * mean
    * mode
    * std
13. is_suspicious
    * max
14. transaction_type	
    * unique

In [98]:
aggregation = {
    'weight': {
        'min_weight': 'min',
        'max_weight': 'max',
        'avg_weight': 'mean',    
    },
    'vsize': {
        'min_vsize': 'min',
        'max_vsize': 'max',
        'avg_vsize': 'mean',
    },
    'inputs_count':{
        'min_inputs_count': 'min',
        'max_inputs_count': 'max',
        'avg_inputs_count': 'mean',
    },
    'outputs_count':{
        'min_outputs_count': 'min',
        'max_outputs_count': 'max',
        'avg_outputs_count': 'mean',
    },
    'inputs_value':{
        'min_inputs_value': 'min',
        'max_inputs_value': 'max',
        'avg_inputs_value': 'mean',
        'sum_inputs_value': 'sum',
    }
    
    
    
}

In [99]:
result_df[result_df['true_input']==1].groupby(
    ['input_add']
).agg(aggregation).head()

weight                        \
                                         min_weight max_weight avg_weight   
input_add                                                                   
02f082f82aa9a019171863b99bbb02ff66e52954      12719      12719    12719.0   
091b13da27e57cef6fb397df453cf9486db41353      12719      12719    12719.0   
112ByUfJJxKQtHbYMpHBkQWfL3vSED7YT6           117720     117720   117720.0   
112DM8h5pquda5iD4WbDQLNPjoz498Dz7y            65100      65100    65100.0   
112R9kdBtaVebnshyrmEXFwV2NXzcDrFDs           206168     206168   206168.0   

                                                        vsize            \
                                         std_weight min_vsize max_vsize   
input_add                                                                 
02f082f82aa9a019171863b99bbb02ff66e52954        NaN      3180      3180   
091b13da27e57cef6fb397df453cf9486db41353        NaN      3180      3180   
112ByUfJJxKQtHbYMpHBkQWfL3vSED7YT6              NaN     29430     29430   
112DM8h5pquda5iD4WbDQLNPjoz498Dz7y              NaN     16275     16275   
112R9kdBtaVebnshyrmEXFwV2NXzcDrFDs              NaN     51542     51542   

                                                                 inputs_count  \
                                         avg_vsize std_vsize min_inputs_count   
input_add                                                                       
02f082f82aa9a019171863b99bbb02ff66e52954    3180.0       NaN                3   
091b13da27e57cef6fb397df453cf9486db41353    3180.0       NaN                3   
112ByUfJJxKQtHbYMpHBkQWfL3vSED7YT6         29430.0       NaN              199   
112DM8h5pquda5iD4WbDQLNPjoz498Dz7y         16275.0       NaN              110   
112R9kdBtaVebnshyrmEXFwV2NXzcDrFDs         51542.0       NaN              349   

                                                                ...         \
                                         max_inputs_count       ...          
input_add                                                       ...          
02f082f82aa9a019171863b99bbb02ff66e52954                3       ...          
091b13da27e57cef6fb397df453cf9486db41353                3       ...          
112ByUfJJxKQtHbYMpHBkQWfL3vSED7YT6                    199       ...          
112DM8h5pquda5iD4WbDQLNPjoz498Dz7y                    110       ...          
112R9kdBtaVebnshyrmEXFwV2NXzcDrFDs                    349       ...          

                                             outputs_count                    \
                                         min_outputs_count max_outputs_count   
input_add                                                                      
02f082f82aa9a019171863b99bbb02ff66e52954                87                87   
091b13da27e57cef6fb397df453cf9486db41353                87                87   
112ByUfJJxKQtHbYMpHBkQWfL3vSED7YT6                       2                 2   
112DM8h5pquda5iD4WbDQLNPjoz498Dz7y                       1                 1   
112R9kdBtaVebnshyrmEXFwV2NXzcDrFDs                       1                 1   

                                                                              \
                                         avg_outputs_count std_outputs_count   
input_add                                                                      
02f082f82aa9a019171863b99bbb02ff66e52954              87.0               NaN   
091b13da27e57cef6fb397df453cf9486db41353              87.0               NaN   
112ByUfJJxKQtHbYMpHBkQWfL3vSED7YT6                     2.0               NaN   
112DM8h5pquda5iD4WbDQLNPjoz498Dz7y                     1.0               NaN   
112R9kdBtaVebnshyrmEXFwV2NXzcDrFDs                     1.0               NaN   

                                                               inputs_value  \
                                         sum_outputs_count min_inputs_value   
input_add                                                                     
02f082f82aa9a019171863b99bbb02ff66

In [93]:
result_df.head()

,txid,confirmations,time,is_coinbase,is_double_spend,is_sw_tx,weight,vsize,inputs_count,outputs_count,...,transaction_type,input_add,output_add,curr_value,tot_prev_tx_value,tot_next_tx_value,diff_prev_time,diff_next_time,true_input,true_output
0,8a06b444098fd4101cc4b12c671299879d06ed7046be50...,14259,1550131695,0,0,0,760,190,1,1,...,1,1JqmxxswaEpvEcLJJLe8BTvmSHKYzY7LBn,1DgSidKHgUPt1CAXHDyjYAeFevG4vdcUmT,1046350184,1372369624,1.046331e+09,0,266178.0,1,1
1,6e5019657ca158a353036407fd6017cb09e055ee0afa24...,14510,1549978917,0,0,0,768,192,1,1,...,1,1JqmxxswaEpvEcLJJLe8BTvmSHKYzY7LBn,14yam1izffm8aA7sD7QDKhzoGHhVicndvx,2252817984,2257975497,2.928215e+09,1190,1844.0,1,1
2,b0186a5d3d9c3ef14c298b447211e2f44ccf80ddd700c8...,16456,1548866324,0,0,0,764,191,1,1,...,1,1JqmxxswaEpvEcLJJLe8BTvmSHKYzY7LBn,1LroCEpTfVcH1XZQTDKyjZrq4BHM1MSi7B,200000000,1234161900,1.574085e+09,91573,12588.0,1,1
3,277092470b6e2bf6f997928f9357d57268ff794a512de4...,16622,1548768572,0,0,0,768,192,1,1,...,1,1JqmxxswaEpvEcLJJLe8BTvmSHKYzY7LBn,1BXG6uVHujSeqUT1Ucv85BTJSxhYokzw8w,200000000,558906374,1.574085e+09,68125,110340.0,1,1
4,be1ada35efe983d9b72e3c3cf3a4e8e08554895771953f...,18094,1547895248,0,0,0,764,191,1,1,...,1,1JqmxxswaEpvEcLJJLe8BTvmSHKYzY7LBn,1BP6vB3Nj1heN2aeUm9BAKj9NcRFZEtVLp,1613000000,1821623317,3.036980e+09,48781,3356.0,1,1


In [ ]:
#отдельно посчитать уникальность адресов относительно транзакций

## Graph

In [ ]:
result_df_dict = result_df.to_dict(orient='index')
not_attrs = ['input_add', 'output_add', 'txid', 'confirmations', 'time']
attrs = [x for x in cols if x not in not_attrs]

In [43]:
G = nx.DiGraph()

In [44]:
%%time
for i in result_df_dict:
    input_add = result_df_dict[i]['input_add']
    output_add = result_df_dict[i]['output_add']
        
    G.add_edge(input_add, output_add, 
               time=result_df_dict[i]['time'],
               is_coinbase=result_df_dict[i]['is_coinbase'],
               is_double_spend=result_df_dict[i]['is_double_spend'],
               weight=result_df_dict[i]['weight'],
               vsize=result_df_dict[i]['vsize'],
               inputs_count=result_df_dict[i]['inputs_count'],
               outputs_value=result_df_dict[i]['outputs_value'],
               max_prev_val=result_df_dict[i]['max_prev_val'],
               min_prev_val=result_df_dict[i]['min_prev_val'],
               avg_prev_val=result_df_dict[i]['avg_prev_val'],
               max_next_val=result_df_dict[i]['max_next_val'],
               min_next_val=result_df_dict[i]['min_next_val'],
               avg_next_val=result_df_dict[i]['avg_next_val'],
               diff_max_prev_time=result_df_dict[i]['diff_max_prev_time'],
               diff_min_prev_time=result_df_dict[i]['diff_min_prev_time'],
               diff_avg_prev_time=result_df_dict[i]['diff_avg_prev_time'],
               diff_max_next_time=result_df_dict[i]['diff_max_next_time'],
               diff_min_next_time=result_df_dict[i]['diff_min_next_time'],
               diff_avg_next_time=result_df_dict[i]['diff_avg_next_time'],
               is_suspicious=result_df_dict[i]['is_suspicious'],
               curr_value=result_df_dict[i]['curr_value'],
               tot_prev_tx_value=result_df_dict[i]['tot_prev_tx_value'],
               tot_next_tx_value=result_df_dict[i]['tot_next_tx_value'],
               diff_prev_time=result_df_dict[i]['diff_prev_time'],
               diff_next_time=result_df_dict[i]['diff_next_time'],
              )

CPU times: user 63.2 ms, sys: 4.47 ms, total: 67.7 ms
Wall time: 66.3 ms
